In [1]:
!pip install datasets
!pip install transformers
!pip install datasets

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
from sklearn.model_selection import train_test_split
import numpy as np
from dataclasses import dataclass
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from transformers import AutoConfig, AutoModelForSeq2SeqLM
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn


In [3]:
df = pd.read_csv("/kaggle/input/arabic-text-summary-final/summarizdataset.csv")
df.shape

(8378, 4)

In [4]:
df

,text,type,Processed Text,summarizer
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,culture,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,culture,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,culture,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...
...,...,...,...,...
8373,\nتأجل الإضراب العام في قطاع الصحة الذي كان مق...,localnews,تاجل الاضراب العام قطاع الصحه مقررا تنفيذه الي...,وكانت انعقدت عشية أمس في وزارة الشؤون الاجتماع...
8374,\nكشف الناشطان كريم نوار وعفيف زقية اللذان أشر...,localnews,كشف الناشطان كريم نوار وعفيف زقيه اشرفا عمليه ...,وأكدا ان تكلفة العلم لم تتجاوز 3 آلاف دينار مق...
8375,\nتمكّنت فرقة الأبحاث والتفتيش للحرس الوطني بط...,localnews,فرقه الابحاث والتفتيش للحرس الوطني بطبلبه ولاي...,كما تمّ إلقاء القبض على عنصر رابع (عمره 32 سنة...
8376,\nقرر الأهالي بمناطق هيشر وعين القارصي والغولا...,localnews,قرر الاهالي بمناطق هيشر وعين القارصي والغولايث...,وتأتي هذه الخطوة على خلفية ما اعتبره أهالي هذه...


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['Processed Text'], df['summarizer'], test_size=0.2, random_state=42)


In [6]:
df_sampled = pd.concat([X_train, Y_train], axis=1)
print(df_sampled.shape)

(6702, 2)


In [7]:
df_sampled = df_sampled.sample(n=1000, random_state=42)
print(df_sampled.shape)

(1000, 2)


In [8]:
df_sampled.to_csv('/kaggle/working/summarize_train_final.csv', index=False)

In [9]:
df = pd.read_csv("/kaggle/working/summarize_train_final.csv")
df

,Processed Text,summarizer
0,قالت وزيره الصحه سميره مرعي فريعه وزارتها تحرص...,\nقالت وزيرة الصحة، سميرة مرعي فريعة، إن وزارت...
1,وضعت عمليه القرعه جرت السبت مالي المنتخب الوطن...,وتعد هذه البطولة القارية ترشيحية لنهائيات بطول...
2,قال النائب حزب صوت الفلاحين فيصل التبيني تونس ...,"وأضاف في مداخلة له اليوم في برنامج الحدث "" تون..."
3,شن تنظيم داعش الارهابي اليوم الاحد هجمات انتحا...,"وقال عيسى: ""وقعت 3 تفجيرات بسيارات مفخخة يقوده..."
4,القت الوحدات الامنيه بقفصه الاحد القبض اشخاص و...,وقد قاموا بتعنيف العون باستعمال قضبان حديدية و...
...,...,...
995,اكدت وزيره الصحه سميره مرعي انه سيتم فتح باب ا...,وأضافت الوزيرة خلال مداخلة لها اليوم في برنامج...
996,اودعت النقابه الوطنيه للصحفيين التونسيين اليوم...,\nأودعت النقابة الوطنية للصحفيين التونسيين الي...
997,تسعى وزاره الهجره الاستراليه اعتماد نظام جديد ...,و من المنتظر أن يتم العمل بهذه التقنية خلال شه...
998,جاءت تونس المرتبه مؤشر افضل الدول تعده مؤسسه ا...,\nجاءت تونس في المرتبة 64 في مؤشر أفضل الدول ا...


In [10]:
d_sampled = pd.concat([X_test, Y_test], axis=1)
print(d_sampled.shape)

(1676, 2)


In [11]:
d_sampled = d_sampled.sample(n=273, random_state=42)
print(d_sampled.shape)

(273, 2)


In [12]:
example_ids = range(0, len(d_sampled))
d_sampled.insert(0, 'example_id', example_ids)
d_sampled

,example_id,Processed Text,summarizer
7121,0,وزير العلاقه الهيئات الدستوريه والمجتمع المدني...,\nعبّر وزير العلاقة مع الهيئات الدستورية والمج...
3590,1,اعلنت شركه هنديه تدعى Ringing Bells اطلاق ارخص...,وأُطلق على الهاتف الجديد اسم Freedom 251، وهو ...
2670,2,اعلن الاتحاد الاوروبي لكره القدم اليوم تشكيله ...,الوسط: بابلو هيرنانديز (سيلتا فيجو)، أندريه هي...
7296,3,اعلنت وزاره التربيه اليوم فتح مناظره بالاختبار...,\nأعلنت وزارة التربية اليوم عن فتح مناظرة خارج...
1545,4,ذكرت صحيفه الغارديان البريطانيه قناه الجزيره ا...,"\nذكرت صحيفة ""الغارديان"" البريطانية أن قناة ""ا..."
...,...,...,...
425,268,هاجم نمر حارسه عندما كانت القفص المخصص حديقه ح...,"وأورد موقع ""سكاي نيوز"" أن الحادث جدّ صباح الاث..."
5507,269,نجح الفريق التونسي المشارك المسابقه العالميه ل...,\nنجح الفريق التونسي المشارك في المسابقة العال...
5144,270,تطرقت الكاتبه الفه يوسف افتتاحيه اليوم الثلاثا...,\nتطرقت الكاتبة ألفة يوسف في افتتاحية اليوم ال...
4561,271,تم العثور الاحد جثه طفله تبلغ العمر سنوات تتدل...,وقد تم نقل الجثة الى المستشفى لعرضها على الطبي...


In [13]:
d_sampled.to_csv('/kaggle/working/summarize_test_final.csv',index = False)

In [14]:
df = pd.read_csv("/kaggle/working/summarize_test_final.csv")
df

,example_id,Processed Text,summarizer
0,0,وزير العلاقه الهيئات الدستوريه والمجتمع المدني...,\nعبّر وزير العلاقة مع الهيئات الدستورية والمج...
1,1,اعلنت شركه هنديه تدعى Ringing Bells اطلاق ارخص...,وأُطلق على الهاتف الجديد اسم Freedom 251، وهو ...
2,2,اعلن الاتحاد الاوروبي لكره القدم اليوم تشكيله ...,الوسط: بابلو هيرنانديز (سيلتا فيجو)، أندريه هي...
3,3,اعلنت وزاره التربيه اليوم فتح مناظره بالاختبار...,\nأعلنت وزارة التربية اليوم عن فتح مناظرة خارج...
4,4,ذكرت صحيفه الغارديان البريطانيه قناه الجزيره ا...,"\nذكرت صحيفة ""الغارديان"" البريطانية أن قناة ""ا..."
...,...,...,...
268,268,هاجم نمر حارسه عندما كانت القفص المخصص حديقه ح...,"وأورد موقع ""سكاي نيوز"" أن الحادث جدّ صباح الاث..."
269,269,نجح الفريق التونسي المشارك المسابقه العالميه ل...,\nنجح الفريق التونسي المشارك في المسابقة العال...
270,270,تطرقت الكاتبه الفه يوسف افتتاحيه اليوم الثلاثا...,\nتطرقت الكاتبة ألفة يوسف في افتتاحية اليوم ال...
271,271,تم العثور الاحد جثه طفله تبلغ العمر سنوات تتدل...,وقد تم نقل الجثة الى المستشفى لعرضها على الطبي...


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:
@dataclass
class LabeledExample:
    paragraph: str
    summary: str

class LabeledDataset(Dataset):
    def __init__(self, data: list[LabeledExample]):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained('moussaKam/AraBART')

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx):
        paragraph, summary = self.data[idx]
        inputs = self.tokenizer.encode_plus(
               paragraph,
               truncation=True,
               padding='max_length',
               max_length=512,
               return_tensors='pt'
              )
        labels = self.tokenizer.encode_plus(
               summary,
               truncation=True,
               padding='max_length',
               max_length=110,
               return_tensors='pt'
              )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': labels['input_ids'].flatten()
            }


In [17]:
# Load and preprocess the labeled dataset
def load_labeled_dataset(file_path, nrows=None):
    df = pd.read_csv(file_path, nrows=nrows)
    dataset = []
    for _, row in df.iterrows():
        paragraph = row['Processed Text']
        summary = row['summarizer']
        dataset.append((paragraph, summary))
    return dataset

In [18]:
# Load the labeled dataset
labeled_dataset = load_labeled_dataset('/kaggle/working/summarize_train_final.csv')
dataset = LabeledDataset(labeled_dataset)

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [19]:
def initialize_model(pretrained_model_name, num_labels, device):

    model_config = AutoConfig.from_pretrained(pretrained_model_name)
    model_config.num_labels = num_labels

    model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name, config=model_config)
    model.to(device)

    return model

device = 'cuda'
model = initialize_model('moussaKam/AraBART', 2, device)


pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [20]:
def create_data_loader(dataset, batch_size):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

def initialize_training_components(model, learning_rate):
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    return optimizer, criterion


batch_size = 8
learning_rate = 5e-5
data_loader = create_data_loader(dataset, batch_size)
optimizer, criterion = initialize_training_components(model, learning_rate)


In [21]:
def prepare_validation_dataset(file_path, tokenizer_name='moussaKam/AraBART'):

    df = pd.read_csv(file_path)
    validation_data = [(row['example_id'], row['Processed Text'],row['summarizer']) for _, row in df.iterrows()]

    class ValidationDataset(Dataset):
        def __init__(self, data, tokenizer):
            self.data = data
            self.tokenizer = tokenizer

        def __len__(self):
            return len(self.data)

        def __getitem__(self, index):
             example_id, paragraph, summarizer = self.data[index]
             inputs = self.tokenizer.encode_plus(paragraph,
                                                 truncation=True,
                                                 padding='max_length',
                                                 max_length=512,
                                                 return_tensors='pt')
             return {
                    'example_id': example_id,
                    'paragraph': paragraph,
                    'summarizer': summarizer,  # Add this line to include the true summary
                    'input_ids': inputs['input_ids'].squeeze(),
                    'attention_mask': inputs['attention_mask'].squeeze()
                   }


    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    dataset_instance = ValidationDataset(validation_data, tokenizer)

    return dataset_instance


file_path = '/kaggle/working/summarize_test_final.csv'
validation_dataset = prepare_validation_dataset(file_path)


In [22]:
def create_data_loader(dataset, batch_size, shuffle):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

batch_size = 8
validation_data_loader = create_data_loader(validation_dataset, batch_size, shuffle=False)


In [23]:
def train_one_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits


        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

def evaluate_model(model, validation_loader, device):
    model.eval()

def save_model(model, directory):
    model.save_pretrained(directory)


num_epochs = 10
for epoch in range(num_epochs):
    train_one_epoch(model, data_loader, optimizer, device)
    evaluate_model(model, validation_data_loader, device)

save_model(model, 'trained_model')


Epoch 1, Loss: 2.3758
Epoch 2, Loss: 0.7849
Epoch 3, Loss: 0.6447
Epoch 4, Loss: 0.5355
Epoch 5, Loss: 0.4680
Epoch 6, Loss: 0.4049
Epoch 7, Loss: 0.3590
Epoch 8, Loss: 0.3195
Epoch 9, Loss: 0.2810


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


Epoch 10, Loss: 0.2452


In [24]:
# Load the trained model
model_path = './trained_model'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [25]:
import csv
import torch

# Function to generate predictions
def generate_predictions(model, data_loader, tokenizer, device):
    model.to(device)
    model.eval()
    predictions = []
    original_word_counts = []

    for batch in data_loader:
        example_ids = batch['example_id'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        paragraphs = batch['paragraph']

        for example_id, inp_id, att_mask, paragraph in zip(example_ids, input_ids, attention_mask, paragraphs):
            paragraph_word_count = len(tokenizer.tokenize(paragraph))
            original_word_counts.append(paragraph_word_count)

            if inp_id.dim() == 1:
                inp_id = inp_id.unsqueeze(0)

            target_length = max(int(0.47 * paragraph_word_count), inp_id.size(1) + 1)
            inp_id = inp_id.to(device)
            att_mask = att_mask.to(device).unsqueeze(0)

            outputs = model.generate(input_ids=inp_id, attention_mask=att_mask, max_length=target_length)
            summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
            predictions.append({'example_id': example_id.item(), 'summary': summary})

    return predictions, original_word_counts

def save_predictions_to_csv(predictions, file_path):
    with open(file_path, 'w', encoding='utf-8', newline='') as file:
        fieldnames = ['example_id', 'summary']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for prediction in predictions:
            writer.writerow(prediction)


tokenizer = AutoTokenizer.from_pretrained('moussaKam/AraBART')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictions, word_counts = generate_predictions(model, validation_data_loader, tokenizer, device)
save_predictions_to_csv(predictions, 'predictions.csv')


In [26]:
def calculate_compression_scores(predictions, original_word_counts, target_ratio_lower, target_ratio_upper):
    compression_ratios = []
    scores = []

    for prediction, word_count in zip(predictions, original_word_counts):
        summary = prediction['summary']
        summary_word_count = len(tokenizer.tokenize(summary))
        compression_ratio = summary_word_count / word_count
        compression_ratios.append(compression_ratio)

        if target_ratio_lower <= compression_ratio <= target_ratio_upper:
            scores.append(1.0)
        else:
            score = max(0, (target_ratio_upper - compression_ratio) / (target_ratio_upper - target_ratio_lower))
            scores.append(score)

    return compression_ratios, scores


target_ratio_lower = 0.3
target_ratio_upper = 0.4
compression_ratios, scores = calculate_compression_scores(predictions, word_counts, target_ratio_lower, target_ratio_upper)

for i, (ratio, score) in enumerate(zip(compression_ratios, scores)):
    print(f"Example ID: {predictions[i]['example_id']}, Compression Ratio: {ratio:.2f}, Score: {score:.2f}")


Example ID: 0, Compression Ratio: 0.46, Score: 0.00
Example ID: 1, Compression Ratio: 0.52, Score: 0.00
Example ID: 2, Compression Ratio: 0.29, Score: 1.14
Example ID: 3, Compression Ratio: 0.89, Score: 0.00
Example ID: 4, Compression Ratio: 0.53, Score: 0.00
Example ID: 5, Compression Ratio: 0.23, Score: 1.69
Example ID: 6, Compression Ratio: 0.16, Score: 2.40
Example ID: 7, Compression Ratio: 0.39, Score: 1.00
Example ID: 8, Compression Ratio: 0.31, Score: 1.00
Example ID: 9, Compression Ratio: 0.82, Score: 0.00
Example ID: 10, Compression Ratio: 0.27, Score: 1.32
Example ID: 11, Compression Ratio: 1.05, Score: 0.00
Example ID: 12, Compression Ratio: 0.79, Score: 0.00
Example ID: 13, Compression Ratio: 0.29, Score: 1.12
Example ID: 14, Compression Ratio: 0.11, Score: 2.87
Example ID: 15, Compression Ratio: 0.10, Score: 2.97
Example ID: 16, Compression Ratio: 0.61, Score: 0.00
Example ID: 17, Compression Ratio: 0.33, Score: 1.00
Example ID: 18, Compression Ratio: 1.15, Score: 0.00
Exa

In [27]:
def summarize_paragraph(paragraph, model, tokenizer, device, ratio=0.47):

    tokens = tokenizer.encode_plus(
        paragraph,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    input_ids, attention_mask = tokens['input_ids'].to(device), tokens['attention_mask'].to(device)

    target_length = max(1, int(ratio * len(tokenizer.tokenize(paragraph))))

    summary_tokens = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=target_length
    )
    summary = tokenizer.decode(summary_tokens[0], skip_special_tokens=True)

    return summary


summary_ratio = 0.60
input_paragraph = "التقنيات الذكية تعزز بشكل كبير التفاعل بين البشر والآلات، وتغير بوضوح نهج الحياة اليومية وطريقة التفكير. فالذكاء الاصطناعي والتعلم الآلي يقدمان حلاً مبتكرًا للتحديات التقنية والاقتصادية، مما يؤدي إلى تطوير العديد من الصناعات وتحسين الخدمات. وبفضل التطور المستمر في هذا المجال، نرى ثورة رقمية تحدث تغييرات عميقة في كافة جوانب الحياة، من العمل إلى الترفيه والتواصل الاجتماعي."
generated_summary = summarize_paragraph(input_paragraph, model, tokenizer, device, summary_ratio)
print("Generated Summary:", generated_summary)


Generated Summary: وبفضل التطور المستمر في هذا المجال، نرى ثورة رقمية تحدث تغييرات عميقة في كافة جوانب الحياة، من العمل إلى الترفيه والتواصل الاجتماعي. فالذكاء الاصطناعي والتعلم الآلي يقدمان حلا مبتكرا


In [29]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu

def calculate_rouge(predictions, true_summaries):
    rouge = Rouge()
    scores = {'rouge-1': [], 'rouge-2': [], 'rouge-l': []}

    for prediction, true_summary in zip(predictions, true_summaries):
        score = rouge.get_scores(prediction['summary'], true_summary)
        scores['rouge-1'].append(score[0]['rouge-1']['f'])
        scores['rouge-2'].append(score[0]['rouge-2']['f'])
        scores['rouge-l'].append(score[0]['rouge-l']['f'])

    avg_scores = {key: sum(value) / len(value) for key, value in scores.items()}
    return avg_scores

def calculate_bleu(predictions, true_summaries):
    bleu_scores = []

    for prediction, true_summary in zip(predictions, true_summaries):
        reference = [true_summary.split()]  
        candidate = prediction['summary'].split()
        score = sentence_bleu(reference, candidate)
        bleu_scores.append(score)

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu_score


true_summaries = [item['summarizer'] for _, item in enumerate(validation_dataset)]

rouge_scores = calculate_rouge(predictions, true_summaries)
bleu_score = calculate_bleu(predictions, true_summaries)

print(f"ROUGE-1 Score: {rouge_scores['rouge-1']:.2f}")
print(f"ROUGE-2 Score: {rouge_scores['rouge-2']:.2f}")
print(f"ROUGE-L Score: {rouge_scores['rouge-l']:.2f}")
print(f"BLEU Score: {bleu_score:.2f}")


ROUGE-1 Score: 0.51
ROUGE-2 Score: 0.38
ROUGE-L Score: 0.50
BLEU Score: 0.29


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [31]:
# Load the pretrained model
pretrained_model = AutoModelForSeq2SeqLM.from_pretrained('moussaKam/AraBART')

# Evaluate the pretrained model
pretrained_predictions, _ = generate_predictions(pretrained_model, validation_data_loader, tokenizer, device)

# Calculate ROUGE and BLEU scores for the pretrained model
pretrained_rouge_scores = calculate_rouge(pretrained_predictions, true_summaries)
pretrained_bleu_score = calculate_bleu(pretrained_predictions, true_summaries)

# Display the scores
print(f"Pretrained ROUGE-1 Score: {pretrained_rouge_scores['rouge-1']:.2f}")
print(f"Pretrained ROUGE-2 Score: {pretrained_rouge_scores['rouge-2']:.2f}")
print(f"Pretrained ROUGE-L Score: {pretrained_rouge_scores['rouge-l']:.2f}")
print(f"Pretrained BLEU Score: {pretrained_bleu_score:.2f}")


Pretrained ROUGE-1 Score: 0.36
Pretrained ROUGE-2 Score: 0.16
Pretrained ROUGE-L Score: 0.35
Pretrained BLEU Score: 0.14
